In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import sqlalchemy
import sqlite3
from subprocess import check_output
import os
%load_ext sql
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

In [3]:
movies = pd.read_csv('/kaggle/input/imdb-extensive-dataset/IMDb movies.csv', index_col = "imdb_title_id")
#print(movies.info())
movies.head(2)

,title,original_title,year,date_published,genre,duration,country,language,director,writer,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
imdb_title_id,,,,,,,,,,,,,,,,,,,,,
tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,NaN,Charles Tait,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,537,$ 2250,NaN,NaN,NaN,7.0,7.0
tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.9,171,NaN,NaN,NaN,NaN,4.0,2.0


In [4]:
names = pd.read_csv('/kaggle/input/imdb-extensive-dataset/IMDb names.csv', index_col = "imdb_name_id")
names['divorces'] = names['divorces'].replace({2:1, 3:1, 4:1, 5:1, 6:1, 7:1, 8:1, 12:1})  ## xoa
names['divorces'] = names['divorces'].replace({0: 'Non-divorces', 1: 'divorces'})
print(names.info())
names.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 175715 entries, nm0000001 to nm9992999
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   name                   175715 non-null  object 
 1   birth_name             175715 non-null  object 
 2   height                 30080 non-null   float64
 3   bio                    122832 non-null  object 
 4   birth_details          75229 non-null   object 
 5   birth_year             75111 non-null   float64
 6   date_of_birth          75102 non-null   object 
 7   place_of_birth         71122 non-null   object 
 8   death_details          26862 non-null   object 
 9   death_year             26838 non-null   float64
 10  date_of_death          26855 non-null   object 
 11  place_of_death         25036 non-null   object 
 12  reason_of_death        11972 non-null   object 
 13  spouses                175715 non-null  int64  
 14  divorces               175715 

,name,birth_name,height,bio,birth_details,birth_year,date_of_birth,place_of_birth,death_details,death_year,date_of_death,place_of_death,reason_of_death,spouses,divorces,spouses_with_children,children,primary_profession,known_for_titles
imdb_name_id,,,,,,,,,,,,,,,,,,,
nm0000001,Fred Astaire,Frederic Austerlitz Jr.,177.0,"Fred Astaire was born in Omaha, Nebraska, to J...","May 10, 1899 in Omaha, Nebraska, USA",1899.0,1899-05-10,"Omaha, Nebraska, USA","June 22, 1987 in Los Angeles, California, USA ...",1987.0,1987-06-22,"Los Angeles, California, USA",pneumonia,2,Non-divorces,1,2,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0072308,tt0043044"
nm0000002,Lauren Bacall,Betty Joan Perske,174.0,Lauren Bacall was born Betty Joan Perske on Se...,"September 16, 1924 in The Bronx, New York City...",1924.0,1924-09-16,"The Bronx, New York City, New York, USA","August 12, 2014 in New York City, New York, US...",2014.0,2014-08-12,"New York City, New York, USA",stroke,2,divorces,2,3,"actress,soundtrack","tt0037382,tt0038355,tt0117057,tt0071877"


In [5]:
ratings = pd.read_csv('/kaggle/input/imdb-extensive-dataset/IMDb ratings.csv')
print(ratings.info())
ratings.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81273 entries, 0 to 81272
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   imdb_title_id              81273 non-null  object 
 1   weighted_average_vote      81273 non-null  float64
 2   total_votes                81273 non-null  int64  
 3   mean_vote                  81273 non-null  float64
 4   median_vote                81273 non-null  float64
 5   votes_10                   81273 non-null  int64  
 6   votes_9                    81273 non-null  int64  
 7   votes_8                    81273 non-null  int64  
 8   votes_7                    81273 non-null  int64  
 9   votes_6                    81273 non-null  int64  
 10  votes_5                    81273 non-null  int64  
 11  votes_4                    81273 non-null  int64  
 12  votes_3                    81273 non-null  int64  
 13  votes_2                    81273 non-null  int

,imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
0,tt0000574,6.1,537,6.3,6.0,54,17,55,121,122,...,6.0,19.0,6.6,14.0,6.3,64.0,6.0,89.0,6.2,309.0
1,tt0001892,5.9,171,6.1,6.0,5,6,17,41,52,...,5.8,4.0,6.5,8.0,5.9,29.0,6.2,27.0,6.0,114.0


In [6]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:////IMDB', echo=False)
movies.to_sql('movies', con = engine)
names.to_sql('names', con = engine)
ratings.to_sql('ratings', con = engine)

# PRACTING

In this chapter you'll learn how to sort and group your results to gain further insight.

## 1. ORDER BY __ DESC 

In SQL, the **ORDER BY** `keyword` is used to sort results in `ascending` or `descending order` according to the values of one or more columns.

By **`default ORDER BY`** will sort in **`ascending`** order.

In [7]:
pd.read_sql(
    """
        SELECT title, original_title, year
        FROM movies
        ORDER BY year
        LIMIT 9
        """, con = engine)

,title,original_title,year
0,The Story of the Kelly Gang,The Story of the Kelly Gang,1906
1,Den sorte drøm,Den sorte drøm,1911
2,L'Inferno,L'Inferno,1911
3,Oborona Sevastopolya,Oborona Sevastopolya,1911
4,Karadjordje,Karadjordje,1911
5,Cleopatra,Cleopatra,1912
6,"From the Manger to the Cross; or, Jesus of Naz...","From the Manger to the Cross; or, Jesus of Naz...",1912
7,Independenta Romaniei,Independenta Romaniei,1912
8,Richard III,Richard III,1912


If you want to sort the results in descending order, you can use the DESC keyword. 

In [8]:
pd.read_sql(
    """
        SELECT title, director
        FROM movies
        WHERE director IS NOT NULL
        ORDER BY director DESC
        """, con = engine)

,title,director
0,Stella í orlofi,Þórhildur Þorleifsdóttir
1,Dalalíf,Þráinn Bertelsson
2,Jón Oddur & Jón Bjarni,Þráinn Bertelsson
3,Nýtt líf,Þráinn Bertelsson
4,Löggulíf,Þráinn Bertelsson
...,...,...
81195,Loafer,A. Bhimsingh
81196,Naya Din Nai Raat,A. Bhimsingh
81197,Pasamalar,A. Bhimsingh
81198,City Dragon,'Philthy' Phil Phillips


This query returns the name of director sorted alphabetically; starting at letter `B` and ending at `S (Smith)`

## Another practing_example

### Example 1.1. Print the title of films released in 2000 or 2012, in the order they were released.

In [9]:
pd.read_sql(
    """
    SELECT title
    FROM movies
    WHERE year IN (2000, 2012)
    ORDER BY year
    """, con = engine)

,title
0,The Fantasticks
1,For the Cause
2,Gang
3,Karobaar: The Business of Love
4,Faa yeung nin wa
...,...
3829,Manam Kothi Paravai
3830,The Adventures of Panda Warrior
3831,Banat El am
3832,Titanic Review


### Example 1.2. Get all details for the most 5 longest duration films except those released in 2015.

In [10]:
pd.read_sql(
    """
    SELECT *
    FROM movies
    WHERE year != 2015
    ORDER BY duration DESC
    LIMIT 5
    """, con = engine)

,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt2008009,The Clock,The Clock,2010,2010-10-15,Drama,3360,UK,English,Christian Marclay,...,"Rosanna Arquette, Bette Davis, Leonardo DiCapr...",Scenes from various films and TV programs that...,8.8,230,None,None,None,NaN,4.0,NaN
1,tt9047474,La flor,La flor,2018,2018-09-21,"Drama, Fantasy, Musical",808,Argentina,"Spanish, French, English, Russian, German, Swe...",Mariano Llinás,...,"Elisa Carricajo, Valeria Correa, Pilar Gamboa,...","A film in six episodes, connected by the same ...",7.9,216,None,$ 4032,$ 4032,77.0,1.0,14.0
2,tt0246135,"Out 1, noli me tangere","Out 1, noli me tangere",1971,2015-11-18,"Drama, Mystery",729,France,"French, English","Jacques Rivette, Suzanne Schiffman",...,"Michèle Moretti, Hermine Karagheuz, Karen Puig...","Following the May 1968 civil unrest in France,...",7.9,1009,None,$ 31539,$ 37743,87.0,11.0,30.0
3,tt0073460,Noce i dnie,Noce i dnie,1975,1975-09-23,"Drama, Romance, War",632,Poland,Polish,Jerzy Antczak,...,"Jadwiga Baranska, Jerzy Binczycki, Emir Buczac...","An epic saga of the Niechcic family, told from...",7.4,348,None,None,None,NaN,2.0,1.0
4,tt0270784,Khleb - imya sushchestvitelnoe,Khleb - imya sushchestvitelnoe,1988,1988-10-06,"Drama, History, Romance",580,Soviet Union,Russian,Grigori Nikulin,...,"Nikolai Voloshin, Andrey Dudarenko, Sergey Nik...",Epic film about life of three generations in c...,5.8,156,$ 60000000,None,None,NaN,NaN,NaN


### Example 1.3. Get the title and gross earnings for movies which begin with the letter 'T' and order the results alphabetically.

In [11]:
pd.read_sql(
    """
    SELECT title, usa_gross_income
    FROM movies
    WHERE (title LIKE 'T%') AND (usa_gross_income IS NOT NULL) 
    ORDER BY title
    """, con = engine)

,title,usa_gross_income
0,T-Rex: Back to the Cretaceous,$ 53354743
1,T2 Trainspotting,$ 2402004
2,THX 1138,$ 2437000
3,TMNT,$ 54149098
4,TRON,$ 33000000
...,...,...
2886,Two-Bit Waltz,$ 2540
2887,Tyler Perry's Boo 2! A Madea Halloween,$ 47319572
2888,Tyrannosaur,$ 22321
2889,Tyrel,$ 21456


### Example 1.4. Get the `allgenders_18age_avg_vote` and `film ID` for every film from the `ratings` table, sorted from highest to lowest score.

In [12]:
pd.read_sql(
    """
    SELECT imdb_title_id, allgenders_18age_avg_vote
    FROM ratings
    ORDER BY allgenders_18age_avg_vote
    """, con = engine)

,imdb_title_id,allgenders_18age_avg_vote
0,tt0017519,NaN
1,tt0018413,NaN
2,tt0019726,NaN
3,tt0020534,NaN
4,tt0020661,NaN
...,...,...
81268,tt6463320,10.0
81269,tt8832158,10.0
81270,tt9279362,10.0
81271,tt9465532,10.0


### Example 1.5. Get the title in reverse order and duration for every film, in order of longest duration to shortest.


In [13]:
pd.read_sql(
    """
    SELECT title, duration, year
    FROM movies
    WHERE year BETWEEN 2009 AND 2012
    ORDER BY duration DESC, title DESC
    """, con = engine)

,title,duration,year
0,The Clock,3360,2010
1,Siglo ng pagluluwal,360,2011
2,"Florentina Hubaldo, CTE",360,2012
3,Gangs of Wasseypur,321,2012
4,Mistérios de Lisboa,272,2010
...,...,...,...
9431,Journey to Mecca,45,2009
9432,Hotarubi no mori e,45,2011
9433,Hibakusha,45,2012
9434,Bai lu yuan,45,2011


### Example 1.6. Get the `birth_year` and `name` of people in the `names` table, in order of 
1) when they were born and 

2) alphabetically by name; 

conditional on the people borned after 1960

In [14]:
pd.read_sql(
    """
    SELECT birth_year, name
    FROM names
    WHERE (birth_year IS NOT NULL) AND (birth_year > 1960)
    ORDER BY birth_year DESC, name
    """, con = engine)

,birth_year,name
0,2010.0,Ariella Nurkovic
1,2009.0,Marta Timofeeva
2,2008.0,Aybars Kartal Özson
3,2007.0,Jeté Laurence
4,2007.0,Luke Judy
...,...,...
30552,1961.0,Zhan Baizhanbayev
30553,1961.0,Zuzana Bydzovská
30554,1961.0,k.d. lang
30555,1961.0,Éva Magyar


### Example 1.7. Get `metascore, release years, language` and `titles` of films ordered by `language` (alphabetically) and `release year`.

In [15]:
pd.read_sql(
    """
    SELECT title, metascore, year, language
    FROM movies
    WHERE (metascore IS NOT NULL) AND (language IS NOT NULL)
    ORDER BY language, year
    """, con = engine)

,title,metascore,year,language
0,Tanna,75.0,2015,Aboriginal
1,Rabbit-Proof Fence,80.0,2002,"Aboriginal, English"
2,Ten Canoes,82.0,2006,"Aboriginal, English"
3,Charlie's Country,75.0,2013,"Aboriginal, English"
4,The Interpreter,62.0,2005,"Aboriginal, English, French, Portuguese"
...,...,...,...,...
12695,Five Fingers for Marseilles,73.0,2017,"Xhosa, Southern Sotho, English, Afrikaans"
12696,Menashe,81.0,2017,"Yiddish, English, Spanish"
12697,The Price,61.0,2017,"Yoruba, English"
12698,White Wedding,60.0,2009,"Zulu, Afrikaans, Xhosa, English"


### Example 1.8. Get the language and average_votes of films in each language made. Then print out the 10 highest average_votes

In [16]:
pd.read_sql(
    """
    SELECT language, AVG(avg_vote)
    FROM movies
    WHERE language IS NOT NULL
    GROUP BY language
    ORDER BY AVG(avg_vote) DESC
    LIMIT 10
    """, con = engine)

,language,AVG(avg_vote)
0,"Hindi, Marathi, Malayalam",9.2
1,"English, Italian, Spanish, Latin, Sicilian",9.0
2,"English, Quenya, Old English, Sindarin",8.9
3,"English, Hebrew, German, Polish",8.9
4,"Malayalam, Hindi, Telugu, Kannada",8.8
5,"Himachali, Hindi",8.8
6,"Bulgarian, Italian",8.8
7,"Tamil, Hindi, English, Telugu",8.7
8,"English, Tamil, Telugu, Hindi",8.7
9,"English, Sindarin, Old English",8.7


## 2. GROUP BY (performing by group)

Now we know how to sort results! 

Often we'll need to aggregate results. 

The simpliest case of groupby is `binary_classes`; such as `male or female`; `divorces or not`, `married or not`, and so on. For instance,
> We want to count the number of `male` and `female` employees in your company. Here, what you want is to group all the males together and count them, and group all the females together and count them. 
>
> We want to count the number of `divorces` employees. Likewise, firstly is grouping all the `divorces cases` together and count them, and group all the `non-divorces` together and count them

In [17]:
pd.read_sql(
    """
        SELECT divorces, COUNT(*) AS count
        FROM names
        GROUP BY divorces
        ORDER BY count DESC
        """, con = engine)

,divorces,count
0,Non-divorces,164268
1,divorces,11447


In SQL, **GROUP BY** allows you to group a result by one or more columns.

For example, we want to explore the quantity from column `reasons_of_death` of the actors; 

In [18]:
pd.read_sql(
    """
        SELECT reason_of_death, COUNT(*) AS count
        FROM names
        WHERE reason_of_death IS NOT NULL
        GROUP BY reason_of_death
        ORDER BY count DESC
        """, con = engine)

,reason_of_death,count
0,heart attack,1813
1,cancer,1558
2,natural causes,708
3,heart failure,488
4,lung cancer,433
...,...,...
1577,AIDS-related lymphoma,1
1578,AIDS-related hepatitis,1
1579,AIDS-related complications,1
1580,AIDS-related cancer,1


### HAVING condition

In SQL, `aggregate functions` can't be used in **WHERE clauses**. For example, the following query is invalid:

In [19]:
pd.read_sql(
    """
        SELECT year, COUNT(*)
        FROM movies
        GROUP BY year
        WHERE COUNT(title) > 10
    """, con = engine)

OperationalError: (sqlite3.OperationalError) near "WHERE": syntax error
[SQL: 
        SELECT year, COUNT(*)
        FROM movies
        GROUP BY year
        WHERE COUNT(title) > 10
    ]
(Background on this error at: http://sqlalche.me/e/e3q8)

This means that **if you want to filter based on the result of an aggregate function, you need another way**! 

That's where the **HAVING clause** comes in. For example,

In [20]:
pd.read_sql(
    """
        SELECT year
        FROM movies
        GROUP BY year
        HAVING COUNT(title) > 200
    """, con = engine)

,year
0,1931
1,1932
2,1933
3,1934
4,1935
...,...
84,2015
85,2016
86,2017
87,2018


The preceding code meant "In how many different years were more than 200 movies released?" So, that is 89 years!

Next, how about the years were more than 500 movies released!

In [21]:
pd.read_sql(
    """
        SELECT year, COUNT(*) as count_films
        FROM movies
        GROUP BY year
        HAVING COUNT(title) > 500
        ORDER BY count_films DESC
    """, con = engine)

,year,count_films
0,2017,3106
1,2016,3033
2,2015,2903
3,2018,2880
4,2014,2851
5,2013,2749
6,2012,2517
7,2011,2389
8,2009,2277
9,2010,2253


Hence, look at this table, we can see that only 53 years has the number_of_films over 500.

**Remark. a GROUP BY clause is required before HAVING. So remember that do not using HAVING clause without GROUP BY keyword before** 

For example, `"how many people were over 88 years old in table names"`

In [22]:
pd.read_sql(
    """ 
    SELECT name, birth_year, death_year,
           (death_year - birth_year)
    FROM names
    WHERE birth_year IS NOT NULL
    AND death_year IS NOT NULL
    HAVING (death_year - birth_year) > 88
    """, con = engine)

OperationalError: (sqlite3.OperationalError) a GROUP BY clause is required before HAVING
[SQL:  
    SELECT name, birth_year, death_year,
           (death_year - birth_year)
    FROM names
    WHERE birth_year IS NOT NULL
    AND death_year IS NOT NULL
    HAVING (death_year - birth_year) > 88
    ]
(Background on this error at: http://sqlalche.me/e/e3q8)

but in this case, the solution is very simple!

In [24]:
pd.read_sql(
    """ 
    SELECT name, birth_year, death_year,
           (death_year - birth_year) AS age
    FROM names
    WHERE birth_year IS NOT NULL
    AND death_year IS NOT NULL
    AND (death_year - birth_year) > 88
    """, con = engine)

,name,birth_year,death_year,age
0,Lauren Bacall,1924.0,2014.0,90.0
1,Ingmar Bergman,1918.0,2007.0,89.0
2,Doris Day,1922.0,2019.0,97.0
3,Marlene Dietrich,1901.0,1992.0,91.0
4,Joan Fontaine,1917.0,2013.0,96.0
...,...,...,...,...
3038,Bibhutibhusan Mukhopadhyay,1894.0,1987.0,93.0
3039,Stephen Gilbert,1912.0,2010.0,98.0
3040,Albert Antalffy,1924.0,2013.0,89.0
3041,Kurt Marti,1921.0,2017.0,96.0
